<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/GPT_%EB%B6%80%EC%A0%95_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EA%B0%95(%ED%81%AC%EB%A1%A4%EB%A7%81_ocncat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) 챗 GPT 크롤링 코드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
# DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cpu'

In [ ]:
import pandas as pd
import re
import tqdm

# DATA_PATH = '/Users/ryuyeseul/Desktop/'

yogiyo = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv")
posneg = pd.read_csv(f"{DATA_PATH}posneg_train_37000.csv")
yogiyo = yogiyo[['고객리뷰','별점','사장댓글']]
yogiyo.columns = ['review','rating','reply']
posneg
posneg = posneg[['sentiment','score']]
train = pd.concat([yogiyo, posneg],axis=1)
train

review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")



In [ ]:
train['reply'] = reply_list
train.head()
mask = (train['review'].str.len() > 10 ) & (train['rating'] == 5) & (train['sentiment'] == 1) & (train['score'] > 0.7 )
df = train[mask]
df.shape
# 챗 gpt로 부정문 가져오기

pos_list = df['review']

In [ ]:
df


,review,rating,reply,sentiment,score
6,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,5,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",1,0.778149
12,너무 맛있게 잘 먹었습니다!,5,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,1,0.745087
21,너무 배고파서 먹다가 찍었어요. 너무맛있어요,5,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,1,0.869636
25,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,5,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,1,0.869688
31,푸짐하고 너무 맛있어용~,5,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,1,0.733229
...,...,...,...,...,...
37124,프랜차이즈답게 깔끔하게 잘 도착했습니다 맛있어요 전부 리뷰서비스로 연유꽃빵 받았는데...,5,고객님 안녕하세요 짬뽕관입니다 D 만족스러운 식사 되신 거 같아 기분 좋아지네요~ ...,1,0.833581
37125,짬뽕이랑 탕수육 정말 맛있어요,5,고객님 안녕하세요 짬뽕관입니다 맛있게 드셔주셔서 감사합니다~ 다음에 주문 주셔도 맛...,1,0.719405
37128,정말 맛있습니다. 감사합니다.,5,고객님 안녕하세요 짬뽕관입니다! 0 음식에 만족하신 거 같아 다행입니다! 다음에 주...,1,0.874910
37137,정말 맛있게 잘먹었어요!자주 시켜 먹을께요!,5,고객님 안녕하세요 짬뽕관입니다~ 맛있게 드셨다는 말씀에 힘이 나는거 같습니다 항상 ...,1,0.833007


## 긍정 -> 부정 리뷰 get

In [ ]:
num = 11

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
import openai

def get_neg_sentence(n,m):
    openai.api_key = 'API'

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': f'{pos_list[n:m]} 이 10개의 긍정적인 리뷰 문장들을, 내용은 동일하되 부정문으로만 바꿔서 순서대로 리스트에 담아줘. ["음식이 다 식어서 와서 눅눅해졌어요.. 실망이에요 다신 안시킬 거 같아요" , ...] 식으로. 10개의 부정문을 모두 잘리지않게 리스트 형태에 담아줘.'
            }
        ]
    )
    globals()[f"postoneg_review_{n}_{m}"] = completion.choices[0].message.content

In [ ]:

from tqdm import tqdm

for i in tqdm(range(num*1000,(num+1)*1000,10)):
    get_neg_sentence(i,i+10)

In [ ]:
# 전체 변환 리뷰의 모양 확인

combined_negative_reviews = []

# for 루프를 사용하여 리뷰를 하나로 결합합니다.
for i in tqdm(range(num*100+1, (num+1)*100+1)):  # 1부터 100까지의 범위를 설정 1~101,  / 1000_1010까지 생성완료 / 1001 / 101, 102
    # 리스트의 이름을 동적으로 생성하여 가져옵니다.
    list_name = f"postoneg_review_{i * 10 - 10}_{i * 10}"  # 리스트 이름 예: postoneg_review_
    current_list = globals()[list_name]  # 동적으로 리스트를 가져옵니다.

    # 현재 리스트를 combined_negative_reviews에 결합합니다.
    combined_negative_reviews.append(current_list)

# 모든 리뷰가 결합된 결과를 확인합니다.
print(combined_negative_reviews)

# filtered_pos_train['neg_review'] = 0



In [ ]:
filtered_pos_train = df
filtered_pos_train['neg_review'] =0

- 추출

In [ ]:
import numpy as np

for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_review_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    reviews = globals()[var_name].split('\n')

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(reviews) ==  10:
        filtered_pos_train['neg_review'].iloc[i:i+10] = reviews

In [ ]:
for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_review_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    reviews_str = globals()[var_name]
    reviews = re.findall(r'"(.*?)"|\'(.*?)\'', reviews_str)

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(reviews) ==  10:
        filtered_pos_train['neg_review'].iloc[i:i+10] = reviews

In [ ]:
filtered_pos_train[num*1000:(num+1)*1000]


## 부정리뷰 -> 사장답변 get

In [ ]:
neg_list = filtered_pos_train['neg_review']

In [ ]:
# 부정변환 문장에 대한 사장님의 답변
def get_neg_reply(n, m):
    openai.api_key = 'API'

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': f'{neg_list[n:m]} 이 10개의 부정적인 리뷰 문장들 각각에 대한 최대한 적절하고, 구체적이고 자연스러운 사장님의 한글 답변 10개를 최대한 길게 생성해서 순서대로 리스트에 담아줘. 각 문장은 " " 사이에, 콤마로 서로 연결시켜줘. 답변만 리스트에 담아줘.'
            }
        ]
    )
    globals()[f"postoneg_reply_{n}_{m}"] = completion.choices[0].message.content

In [ ]:
filtered_pos_train['neg_reply'] = 0


In [ ]:
from tqdm import tqdm

for i in tqdm(range(num*1000,(num+1)*1000,10)):
    get_neg_reply(i,i+10)

In [ ]:
# 전체 변환 리뷰의 모양 확인

combined_negative_replies = []

# for 루프를 사용하여 리뷰를 하나로 결합합니다.
for i in tqdm(range(num*100+1, (num+1)*100+1)):  # 1부터 100까지의 범위를 설정
    # 리스트의 이름을 동적으로 생성하여 가져옵니다.
    list_name = f"postoneg_reply_{i * 10 - 10}_{i * 10}"  # 리스트 이름 예: postoneg_review_
    current_list = globals()[list_name]  # 동적으로 리스트를 가져옵니다.

    # 현재 리스트를 combined_negative_reviews에 결합합니다.
    combined_negative_replies.append(current_list)

# 모든 리뷰가 결합된 결과를 확인합니다.
print(combined_negative_replies)

- 추출

In [ ]:
import numpy as np

for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_reply_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    replies = globals()[var_name].split('\n')

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(replies) ==  10:
        filtered_pos_train['neg_reply'].iloc[i:i+10] = replies


In [ ]:

for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_reply_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    replies_str = globals()[var_name]
    replies = re.findall(r'"(.*?)"|\'(.*?)\'', replies_str)

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(replies) ==  10:
        filtered_pos_train['neg_reply'].iloc[i:i+10] = replies

In [ ]:
mask = (filtered_pos_train['neg_review'] != 0) & (filtered_pos_train['neg_reply'] != 0)
filtered_final_train = filtered_pos_train[mask]

In [ ]:
filtered_final_train
filtered_final_train.to_csv(f"filtered_final_train_10000_11000.csv")

In [ ]:
filtered_final_train

# 2) 파일 concat 및 전처리 코드

In [ ]:
import pandas as pd
import re
import tqdm

# /content/

In [ ]:
850+860+950+775+940+920

5295

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(f'gpt_neg_reviews_4400.csv')

In [ ]:
df

,Unnamed: 0,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,부정,0.908812
1,1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,부정,0.820514
2,2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,부정,0.941290
3,3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,부정,0.910234
4,4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,부정,0.928585
...,...,...,...,...,...,...,...,...
4396,5335,너무 맛있게 잘먹었습니다! 일주일에 세번씩은 꼭 시키는것 같아요~!,앞으로도 초밥이 생각나실 땐 부성초밥 찾아주세요~ 항상 신선하고 맛있는 음식 보내...,5,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,부정,0.931082
4397,5336,단골입니다 일전에 닉네임 nangja 을 안 썼더니 사장님의 요청이 맛있게 먹고 냉...,고객님의 리뷰가 저희에겐 크나 큰 힘이 된답니다 언제나 만족스러운 식사시간 되실 ...,5,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,부정,0.932665
4398,5337,처음 주문 했는데. 너무 맛있어요. 특히 VIP 초밥은 고퀄재료들로 다양하게 구성되...,식사가 만족스러우셨다니 너무 기분이 좋습니다 항상 고객님을 생각하며 신선한 음식만 ...,5,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,부정,0.890285
4399,5338,빠르게 잘 받았습니다 좋아하는 초밥이 다 있네요~ 배달은 딱 먹고 싶은 초밥구성이 ...,"주문해주셔서 정말 감사드립니다 저희 부성초밥은 매순간 최선을 다 하며, 개선해야 할...",5,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,부정,0.921841


In [ ]:
# num0~8,11은 기본 답변 / num9~10은 최대한 길게 답변 요청

gpt_aug_num0 = pd.read_csv("filtered_final_train_1_1000.csv") #850
gpt_aug_num1 = pd.read_csv("filtered_final_train_1000_2000.csv") #860
gpt_aug_num2 = pd.read_csv("filtered_final_train_2000_3000_2.csv") #775
gpt_aug_num3 = pd.read_csv("filtered_final_train_3000_4000.csv") #950
gpt_aug_num4 = pd.read_csv("filtered_final_train_4000_5000.csv") #940

gpt_aug_num5 = pd.read_csv("filtered_final_train_5000_6000.csv") #920
gpt_aug_num6 = pd.read_csv("filtered_final_train_6000_7000.csv") #900
gpt_aug_num7 = pd.read_csv("filtered_final_train_7000_8000.csv") #920
gpt_aug_num8 = pd.read_csv("filtered_final_train_8000_9000.csv") #830
gpt_aug_num9 = pd.read_csv("filtered_final_train_9000_10000.csv") #815
gpt_aug_num10 = pd.read_csv("filtered_final_train_10000_11000.csv") #840
gpt_aug_num11 = pd.read_csv("filtered_final_train_11000_12000.csv") #745



In [ ]:
gpt_aug_num0.shape, gpt_aug_num1.shape, gpt_aug_num2.shape, gpt_aug_num3.shape, gpt_aug_num4.shape, gpt_aug_num5.shape, gpt_aug_num6.shape, gpt_aug_num7.shape, gpt_aug_num8.shape, gpt_aug_num9.shape, gpt_aug_num10.shape, gpt_aug_num11.shape

((850, 7),
 (860, 7),
 (775, 8),
 (950, 7),
 (940, 7),
 (920, 7),
 (900, 8),
 (920, 7),
 (830, 7),
 (815, 7),
 (840, 8),
 (765, 8))

In [ ]:
tmp = pd.concat([gpt_aug_num0,gpt_aug_num1,gpt_aug_num2, gpt_aug_num3, gpt_aug_num4, gpt_aug_num5, gpt_aug_num6, gpt_aug_num7, gpt_aug_num8, gpt_aug_num9, gpt_aug_num10, gpt_aug_num11],axis=0)
tmp

,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,"['김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.',",안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,"'양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.',",배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,"'음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.',",음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,"'잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.',",잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,"'가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.',",가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN
...,...,...,...,...,...,...,...,...
760,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.694974,"('', '맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요.. 다...","""안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망...",32714.0
761,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.697194,"('', '맛도 없고, 배달도 늦어서 너무 실망했습니다')","""고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요...",32739.0
762,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,1,0.692698,"('', '진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요')","""맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으...",32847.0
763,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,1,0.699261,"('', '완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요')","""고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기...",32866.0


In [ ]:
tmp

## 데이터 정제

In [ ]:

review_list = []
for review in tmp['neg_review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", review)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

tmp['neg_review'] = review_list
tmp.head()


,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN


In [ ]:
reply_list = []
for reply in tmp['neg_reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

tmp['neg_reply'] = reply_list
tmp.head()


,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN


- 2차 정제

In [ ]:
review_list = []
for review in tmp['neg_review']:
    if isinstance(review, str):
        pat = r'^\s*,\s*'
        result = re.sub(pat, '', review)
        review_list.append(result)
    else:
        review_list.append("")

tmp['neg_review'] = review_list
tmp.head()

,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN


In [ ]:
reply_list = []
for reply in tmp['neg_reply']:
    if isinstance(reply, str):
        pat = r'^\s*,\s*'
        result = re.sub(pat, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

tmp['neg_reply'] = reply_list
tmp.head()

,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN


In [ ]:
review_list = []
for review in tmp['neg_review']:
    if isinstance(review, str):
        pat = r'^\s*'
        result = re.sub(pat, '', review)
        review_list.append(result)
    else:
        review_list.append("")

tmp['neg_review'] = review_list
tmp.head()

,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN


In [ ]:
reply_list = []
for reply in tmp['neg_reply']:
    if isinstance(reply, str):
        pat = r'^\s*'
        result = re.sub(pat, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

tmp['neg_reply'] = reply_list
tmp.head()

,review,reply,rating,sentiment,score,neg_review,neg_reply,Unnamed: 0
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,NaN
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,NaN
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,NaN
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,NaN
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,NaN


In [ ]:
tmp = tmp.drop(columns = 'Unnamed: 0')
tmp.isna().sum()

review        0
reply         0
rating        0
sentiment     0
score         0
neg_review    0
neg_reply     0
dtype: int64

In [ ]:
tmp

## 데이터 제거
-  -> 9688개

In [ ]:
mask = (tmp['neg_review'].str.len() <= 4) | (tmp['neg_reply'].str.len() <= 4)
# tmp = tmp[~mask]
tmp[mask]

,review,reply,rating,sentiment,score,neg_review,neg_reply
150,진짜 오늘 처음시켜먹었는데 맛있습니다 서비스 많이 주셔서 감사해요 !,사진이. 계속 팍팍 퍼드릴게요 감사합니다~,5,1,0.854442,진짜 오늘 처음시켜먹었는데 별로였어요. 맛도 없고 서비스도 별로였어요.,0
151,배달이 생각보다 빨리 와서 친구 오는거 기다리느라 10분정도 뒤에 먹었는데도 치즈가...,"치즈는 팩퍼 자연산 이구요, 새벽에 따끈따끈하게 받는 생밀떡이랍니다 감사합니다, 자...",5,1,0.880215,치즈는 실망이었어요. 맛이 별로 없고 양도 적어서 아쉬웠어요.,0
152,넘모 맛있어요~약도 적당하고 만족합니다,넘모 5 맛있어서 별 5개 꼭 받도록 하겠습니다. 감사해요,5,1,0.718950,넘모 별로였어요~약이 너무 많아서 만족하지 못했습니다.,0
153,제가 사진을 못찍는건지 먹다가 찍어서그런건지 사진이잘안나와요 그래도 사진과는 다르게...,열기로 한 시간에는 주문 받는게 당연하기에. 좋은평 주시니 감사합니다~,5,1,0.899489,사진과 달라서 실망했어요. 기대하고 먹었는데 맛이 별로였어요.,0
154,리뷰 잘안쓰는데 사장님이 친절하셔서 쓰게 되네요 맛이 인위적이지 않고 맛있네요.중간...,"감사하네요, 떡깨비 때문에 친정을 오시고 너무 부담을 가득 담았더니 . 기대가 큰 ...",5,1,0.879957,친절하지 않아서 실망했어요. 맛도 별로였고 인위적인 느낌이었어요.,0
...,...,...,...,...,...,...,...
750,양도 많고 맛있어요 3분에 1만 덜어서 먹었는데도 많네요.,푸짐하게 드시라고 많이 드리고 있어요 만족스러운 식사였다니 넘 감사합니다 오늘하루...,5,1,0.687433,"음식이 조금 짜서 좀 힘들게 먹었어요, 별로 좋지 않은 경험이었어요",
751,양도. 푸짐하게 주시고 맛있게 잘먹었습니다,감사합니다 언제나 변함없는 맛으로 보답드릴께요 오늘하루 즐겁고 행복하게 보내세요,5,1,0.690949,"음식이 많이 짜서 얼마 못먹을 것 같아요, 너무 짜서 맛이 별로였어요",
752,맛있게 잘 먹었습니다 다음에도 시켜먹을게요,감사합니다 생각날때 종종 찾아주세요 오늘하루 즐겁게 보내세요,5,1,0.694583,"응답도 느리고 음식도 맛이 없어서 실망스러웠어요, 다음에는 시키지 않을 거예요",
753,양도 많고 넘 맛있었어요!,안녕하세요 수 많은 맛집속에 저희를 믿고 주문 주시고 맛있게 드셔주심 진심으로 감사...,5,1,0.695820,"음식이 시끄러워서 집중해서 먹을 수가 없었어요, 다음에는 다른 음식점을 시킬 걸 그...",


In [ ]:
tmp = tmp[~mask]
tmp.shape

(9796, 7)

In [ ]:
index_error = tmp[tmp['neg_reply'].str.contains("답변이 없습니다")].index
tmp.drop(index =index_error, inplace=True )

In [ ]:
tmp.shape

(9688, 7)

## 긍부정 재판별

In [ ]:
tmp

,review,reply,rating,sentiment,score,neg_review,neg_reply
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...
...,...,...,...,...,...,...,...
760,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.694974,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하..."
761,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.697194,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....
762,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,1,0.692698,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...
763,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,1,0.699261,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 129.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 96.8 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline


In [ ]:
import pandas as pd
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name, framework="pt")


In [ ]:
classifier = pipeline("sentiment-analysis", model=model_name)


In [ ]:
classifier

In [ ]:
from tqdm import tqdm  # tqdm 모듈을 import

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):  # tqdm으로 반복문을 감싸서 진행 상황 표시
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = '부정'
        else:
            sentiment = '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    df = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return df


In [ ]:
tmp_posneg = analyze_reviews(tmp['neg_review'])

100%|██████████| 9688/9688 [15:33<00:00, 10.38it/s]


In [ ]:
# tmp_posneg.to_csv("tmp_posneg_0_3000.csv", index=False)
# tmp_posneg.to_csv("tmp_posneg_3000_6000.csv", index=False)
# tmp_posneg.to_csv("tmp_posneg_0_8000.csv", index=False)

tmp_posneg.to_csv("tmp_posneg_0_12000.csv", index=False)



In [ ]:
tmp_posneg.reset_index()

,index,review,sentiment,score
0,0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,부정,0.908812
1,1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,부정,0.820514
2,2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,부정,0.941290
3,3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,부정,0.910234
4,4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,부정,0.928585
...,...,...,...,...
9683,9683,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...",부정,0.942769
9684,9684,"맛도 없고, 배달도 늦어서 너무 실망했습니다",부정,0.912723
9685,9685,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,부정,0.894588
9686,9686,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,부정,0.933854


In [ ]:
tmp

,review,reply,rating,sentiment,score,neg_review,neg_reply
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,1,0.778149,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,1,0.745087,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,1,0.869636,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,1,0.869688,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,1,0.733229,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...
...,...,...,...,...,...,...,...
760,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.694974,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하..."
761,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.697194,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....
762,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,1,0.692698,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...
763,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,1,0.699261,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...


In [ ]:
tmp = tmp.reset_index()
tmp_posneg = tmp_posneg.reset_index()
train = pd.concat([tmp[['review', 'reply', 'rating', 'neg_review', 'neg_reply']], tmp_posneg[['sentiment', 'score']]], axis=1)


In [ ]:
mask = (train['sentiment'] == '부정')
train_ft = train[mask]
train_ft

,review,reply,rating,neg_review,neg_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,부정,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,부정,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,부정,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,부정,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,부정,0.928585
...,...,...,...,...,...,...,...
9683,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",부정,0.942769
9684,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,부정,0.912723
9685,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,부정,0.894588
9686,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,부정,0.933854


In [ ]:
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"


In [ ]:
# 부정 -> 0 으로 치환
train_ft['sentiment'] = 0
print(train_ft['sentiment'].sum()) # 합은 0
train_ft[train_ft['sentiment'] == '부정']

0


<ipython-input-187-fd411b821e81>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['sentiment'] = 0


,review,reply,rating,neg_review,neg_reply,sentiment,score


In [ ]:
train_ft['neg_reply']

KeyError: ignored

In [ ]:
train_ft.columns = ['original_review','original_reply','rating','gpt_review','gpt_reply','sentiment','score']
train_ft

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9683,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9684,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9685,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9686,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [ ]:
train = pd.read_csv("gpt_neg_9400.csv")

In [ ]:
train['gpt_reply']

0       안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...
1       배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...
2       음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...
3       잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...
4       가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...
                              ...                        
9373    안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...
9374    고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....
9375    맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...
9376    고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...
9377    진심으로 죄송합니다. 맛과 배달 속도에 대한 실망을 드린 점 정말 죄송합니다. 저희...
Name: gpt_reply, Length: 9378, dtype: object

In [ ]:
train_ft.to_csv("gpt_neg_9400.csv", index=False)

In [ ]:
mask = (tmp_posneg['score'] < 0.6) & (tmp_posneg['sentiment'] == '부정')
tmp_posneg[mask]

,review,sentiment,score
349,넘 맛있어서 담에 또 시켜먹을 생각이에요. 어제 시켜먹었는데 기대했던 맛이 아니어...,부정,0.587878
983,더 맛있는 곳이 많아요,부정,0.356890
988,다른 음식이 더 맛있었어요,부정,0.402155
990,번창하지 않았으면 좋겠어요,부정,0.568649
1225,오늘도 못 먹었습니다 감사함니다,부정,0.577824
1235,아스파라거스가 맛있어요. 그런데 다른 것들은 별로였어요.,부정,0.528413
1281,다른 샐러드는 별로였어요,부정,0.568358
2973,더 맛있는 집들이 많아서 다음에는 다른 곳에서 시킬 거예요.,부정,0.525947
3334,좀 더 다양한 종류의 메뉴가 있었으면 좋겠어요. 지루하게 느껴져서요.,부정,0.585472
3335,좀 더 고소한 맛이 있었으면 좋겠어요. 현재의 맛은 좀 무겁게 느껴져서요.,부정,0.554153


In [ ]:
total_tmp = pd.concat([tmp,tmp_posneg],axis=1)

InvalidIndexError: ignored

In [ ]:
tmp_posneg_0_3000 = pd.read_csv("tmp_posneg_0_3000.csv")
tmp_posneg_3000_6000 = pd.read_csv("tmp_posneg_0_3000.csv")

In [ ]:
tmp.reset_index(drop=True, inplace=True)
tmp_posneg.reset_index(drop=True, inplace=True)
total_tmp = pd.concat([tmp, tmp_posneg], axis=1, ignore_index=True)

In [ ]:
total_tmp = total_tmp[[0,1,2,5,6,8,9]]
total_tmp.columns = ['original_review','original_reply','rating','gpt_review','gpt_reply','sentiment','score']

KeyError: ignored

In [ ]:
mask = total_tmp['sentiment'] == '부정'  # 130개 정도가 긍정 리뷰  -> 5200개 정도
total_tmp[mask]

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,부정,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,부정,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,부정,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,부정,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,부정,0.928585
...,...,...,...,...,...,...,...
5335,너무 맛있게 잘먹었습니다! 일주일에 세번씩은 꼭 시키는것 같아요~!,앞으로도 초밥이 생각나실 땐 부성초밥 찾아주세요~ 항상 신선하고 맛있는 음식 보내...,5,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,부정,0.931082
5336,단골입니다 일전에 닉네임 nangja 을 안 썼더니 사장님의 요청이 맛있게 먹고 냉...,고객님의 리뷰가 저희에겐 크나 큰 힘이 된답니다 언제나 만족스러운 식사시간 되실 ...,5,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,부정,0.932665
5337,처음 주문 했는데. 너무 맛있어요. 특히 VIP 초밥은 고퀄재료들로 다양하게 구성되...,식사가 만족스러우셨다니 너무 기분이 좋습니다 항상 고객님을 생각하며 신선한 음식만 ...,5,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,부정,0.890285
5338,빠르게 잘 받았습니다 좋아하는 초밥이 다 있네요~ 배달은 딱 먹고 싶은 초밥구성이 ...,"주문해주셔서 정말 감사드립니다 저희 부성초밥은 매순간 최선을 다 하며, 개선해야 할...",5,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,부정,0.921841


In [ ]:
total_tmp[condition2]

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
60,양많고 맛좋고 포장좋고 다 좋아요 잘먹었습니다.,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있는 메뉴들은 매번 양껏 먹어도 매번...,5,양많고 맛좋고 포장좋고 다 좋아요 잘먹었습니다.,감사합니다. 다음에도 기대해주세요!,긍정,0.729621
61,잘받았습니다 감사합니다,고객님에게 맛있는 시간을 제공해드린 것 같아 기분이 뿌듯하네요 저희 1988응답하라...,5,잘받았습니다 감사합니다,고객님의 만족을 위해 더 노력하겠습니다.,긍정,0.735581
62,존맛탱이네요. 여자친구도 좋아하네요,항상 더 나은 브랜드가 되고자 최선을 다하는 1988응답하라추억의옛날도시락입니다! ...,5,존맛탱이네요. 여자친구도 좋아하네요,많은 양과 맛에 만족하셨다니 기쁘네요.,긍정,0.767086
63,빠른배달감사합니다. 양도많고 맛있고 배부르게먹었어요,안녕하세요 oh 님! 소중한 리뷰 남겨 주셔서 감사합니다! 메뉴를 드시다가 양이 모...,5,빠른배달감사합니다. 양도많고 맛있고 배부르게먹었어요,항상 최선을 다하는 원픽입니다.,긍정,0.741651
64,양 진짜많더라구요.미쳤음,안녕하세요 정성 가득한 리뷰 작성해주셔서 감사합니다! 고객님들의 배고픔을 저희는 지...,5,양 진짜많더라구요.미쳤음,많은 양과 좋은 맛을 제공하기 위해 노력하고 있습니다.,긍정,0.788276
...,...,...,...,...,...,...,...
4907,완전 맛있게 잘 먹었습니다 순두부로 변경하니 훨씬 맛있었구 김치의 새콤함이 삼겹살과...,안녕하세요 신자매 김치찜 김치찌개를 찾아주셔서 진심으로 감사합니다 순두부 변경은 완...,5,완전 맛있게 잘 먹었습니다 순두부로 변경하니 훨씬 맛있었구 김치의 새콤함이 삼겹살...,고객님 맛있게 드셨다니 저희도 기쁩니다. 다음에도 항상 맛있는 음식으로 찾아뵐 수...,긍정,0.725545
4908,맛있게 잘 먹었습니다.다음에 또 주문하러 오겠습니다.,안녕하세요 신자매 김치찜 김치찌개를 찾아주셔서 진심으로 감사합니다 맛있게 잘 드시고...,5,맛있게 잘 먹었습니다.다음에 또 주문하러 오겠습니다.,고객님 푸짐하고 맛있었다니 정말 좋아요. 맛있는 음식으로 든든하게 보답하겠습니다....,긍정,0.700835
4909,푸짐하니 한끼먹기 너무좋네요~너무맛납니다 강추!,안녕하세요 신자매 김치찜 김치찌개를 찾아주셔서 진심으로 감사합니다 푸짐하니 한끼 먹...,5,푸짐하니 한끼먹기 너무좋네요~너무맛납니다 강추!,고객님 맛있게 드셨다니 기뻐요. 항상 만족하실 수 있도록 최선을 다하겠습니다. 감...,긍정,0.717526
5016,"오늘도 맛있게 잘 먹었습니다, 감사합니당","반갑습니다~ hj 님, 주문해 주시고 음식사진과 맛있게 드셨다는 후기까지 너무너무 ...",5,"오늘도 맛있게 먹었지만, 다음에는 다른 곳에서 시켜먹을 거예요","맛을 맛있게 먹었지만, 다음 번에는 다른 식당에서 주문하실 것 같다는 의견 감사합...",긍정,0.474395


In [ ]:
condition1 = (total_tmp.index >= (850+860)) & (total_tmp.index < (850+860+820))
condition2 = total_tmp['sentiment'] == '긍정'

mask = condition1 | condition2

total_tmp_excluded = total_tmp[~mask]
total_tmp_excluded

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,부정,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,부정,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,부정,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,부정,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,부정,0.928585
...,...,...,...,...,...,...,...
5335,너무 맛있게 잘먹었습니다! 일주일에 세번씩은 꼭 시키는것 같아요~!,앞으로도 초밥이 생각나실 땐 부성초밥 찾아주세요~ 항상 신선하고 맛있는 음식 보내...,5,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,부정,0.931082
5336,단골입니다 일전에 닉네임 nangja 을 안 썼더니 사장님의 요청이 맛있게 먹고 냉...,고객님의 리뷰가 저희에겐 크나 큰 힘이 된답니다 언제나 만족스러운 식사시간 되실 ...,5,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,부정,0.932665
5337,처음 주문 했는데. 너무 맛있어요. 특히 VIP 초밥은 고퀄재료들로 다양하게 구성되...,식사가 만족스러우셨다니 너무 기분이 좋습니다 항상 고객님을 생각하며 신선한 음식만 ...,5,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,부정,0.890285
5338,빠르게 잘 받았습니다 좋아하는 초밥이 다 있네요~ 배달은 딱 먹고 싶은 초밥구성이 ...,"주문해주셔서 정말 감사드립니다 저희 부성초밥은 매순간 최선을 다 하며, 개선해야 할...",5,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,부정,0.921841


In [ ]:
total_tmp[~condition2]

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,부정,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,부정,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,부정,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,부정,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,부정,0.928585
...,...,...,...,...,...,...,...
5335,너무 맛있게 잘먹었습니다! 일주일에 세번씩은 꼭 시키는것 같아요~!,앞으로도 초밥이 생각나실 땐 부성초밥 찾아주세요~ 항상 신선하고 맛있는 음식 보내...,5,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,부정,0.931082
5336,단골입니다 일전에 닉네임 nangja 을 안 썼더니 사장님의 요청이 맛있게 먹고 냉...,고객님의 리뷰가 저희에겐 크나 큰 힘이 된답니다 언제나 만족스러운 식사시간 되실 ...,5,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,부정,0.932665
5337,처음 주문 했는데. 너무 맛있어요. 특히 VIP 초밥은 고퀄재료들로 다양하게 구성되...,식사가 만족스러우셨다니 너무 기분이 좋습니다 항상 고객님을 생각하며 신선한 음식만 ...,5,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,부정,0.890285
5338,빠르게 잘 받았습니다 좋아하는 초밥이 다 있네요~ 배달은 딱 먹고 싶은 초밥구성이 ...,"주문해주셔서 정말 감사드립니다 저희 부성초밥은 매순간 최선을 다 하며, 개선해야 할...",5,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,부정,0.921841


In [ ]:
total_tmp_excluded.to_csv("gpt_neg_reviews_4400.csv")

In [ ]:
total_tmp[~condition2].to_csv("gpt_neg_reviews_5200.csv")

In [ ]:
df = pd.concat([tmp_posneg_0_3000,tmp_posneg_3000_6000],axis=0)
df

,review,sentiment,score
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,부정,0.908812
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,부정,0.820514
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,부정,0.941290
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,부정,0.910234
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,부정,0.928585
...,...,...,...
2525,"식감이 좋지 않았고, 맛도 그닥이었습니다. 좀 더 발전해야 할 것 같아요.",부정,0.916782
2526,맛이 너무 안나서 실망했어요. 다른 곳에서 먹는 것이 나았을 것 같아요.,부정,0.939609
2527,음식이 너무 맛이 없었어요. 다음에는 더 맛있는 곳에서 시킬 거예요.,부정,0.911358
2528,고민하지 말고 다른 곳에서 시키세요. 여기 음식은 별로에요.,부정,0.797302


In [ ]:
mask = tmp_posneg['sentiment'] == '부정'
tmp_posneg[mask] # 긍정인거 없는 것으로 확인됨

,review,sentiment,score
0,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시...",부정,0.942769
1,양이 너무 적고 서비스도 별로였어요.,부정,0.923027
2,맛이 너무 안 좋아서 먹을 수가 없었어요.,부정,0.916261
3,음식이 너무 짜서 입맛이 맞지 않았어요.,부정,0.921768
4,고기가 많이 불려서 맛이 좀 아쉬웠어요.,부정,0.894002
...,...,...,...
2805,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,부정,0.931082
2806,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,부정,0.932665
2807,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,부정,0.890285
2808,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,부정,0.921841


In [ ]:
tmp_posneg[['sentiment','score']]

,sentiment,score
0,부정,0.942769
1,부정,0.923027
2,부정,0.916261
3,부정,0.921768
4,부정,0.894002
...,...,...
2805,부정,0.931082
2806,부정,0.932665
2807,부정,0.890285
2808,부정,0.921841


In [ ]:
!pip install streamlit

In [ ]:
!streamlit hello #


In [ ]:
import sys

In [ ]:
import pandas as pd

In [ ]:
add_1_train = pd.read_csv('add_filtered_pos_train_0_1000.csv')

In [ ]:
add_2_train = pd.read_csv('add_filtered_pos_train_1000_2000.csv')

In [ ]:
add_2_combined = pd.read_csv('combined_negative_realreviews_11000_12000.csv')

In [ ]:
two_train = pd.read_csv("filtered_pos_train_2000_3000 (1).csv", encoding='utf-8')


In [ ]:
two_reply= pd.read_csv(f"combined_negative_replies_2000_3000.csv")

In [ ]:
eleven_train = pd.read_csv(f"filtered_pos_train_11000_12000.csv")

In [ ]:
eleven_reply = pd.read_csv(f"combined_negative_replies_11000_12000 (2).csv")

In [ ]:
add_2_combined.shape, add_2_train.shape

((100, 1), (1000, 5))

In [ ]:
add_2_train

,original_review,original_reply,rating,gpt_review,neg_reply
0,맛도 괜찮고 양도 충분합니다. 감사합니다.,맛있는 음식은 물론 최고의 서비스로 좋은 기억만 남겨드리겠습니다! 늘 고객님의 의견...,5,맛도 별로고 양도 적어요.,0
1,"배달도 30분이나 일찍오고 양도 완전 혜자였어요 고기양도 진짜 많구, 무엇보다 두부...",오늘도 저희 매장을 찾아주셔서 정말 감사합니다! 저희는 고객만족을 최우선으로 늘 만...,5,포장 상태가 좋지 않았어요.,0
2,양도 엄청 많고 맛있어요! 포장도 꼼꼼하게 잘 해주셨어요,고객님들의 리뷰를 보며 마음이 정말 훈훈해집니다 고객님도 저희 매장과 함께 따뜻하셨...,5,맛도 별로였고 양도 적어요.,0
3,야무지게 맛있어서 또 시켜먹을거같은 깁치찜이네요 감사합니다,고객님의 리뷰에 너무 기분 좋게 하루를 시작했습니다 고객님들의 리뷰에 저희가 할 수...,5,맛없고 양도 적고 포장도 안되어있어요.,0
4,"워 징심 존맛. 깜짝놀랐어요 양도 많고 맛도 미쳤음 진심 배달도 빠르규, 최고에용 !",지나가는 한마디에도 큰 힘이 되는 요즘 고객님이 남겨주신 리뷰에 큰 힘이 됩니다 힘...,5,맛이 없고 양도 적어요.,0
...,...,...,...,...,...
995,다른 가게보단 매운거 같아요. 제일 매운 맛 시켰더니 죽을뻔 했어요 편지 넘 감동이...,안녕하세요 마라쿡입니다! 저희 가게를 찾아주시는 고객님들께서 맛있다고 한마디 남겨주...,5,양이 너무 적고 서비스도 별로였어요.,0
996,아이가 주말마다 먹더니 진짜맛있다고 오늘은 주문해서 먹자고해서 시켰어요 정~말 맛있...,반갑습니다! 소중한 사진리뷰 감사합니다. 즐거운 한끼가 되신것같아 제 기분도 덩달아...,5,맵기가 제 입맛에는 너무 강해서 먹을 수가 없었어요.,0
997,정말 놀라운 경험입니다. 제가 먹어본 마라탕 중에 최고에요. 너무 맛있어요!,안녕하세요! 마라쿡 을 찾아주셔서 감사드려용~ 저희 음식을 맛있게 드셨다니 안심이 ...,5,"주문한 음식이 늦게 도착했고, 푸드포장 상태도 좋지 않았어요.",0
998,배달도 빨리오고 맛있어요 저번에 시켜먹었는데 또 시켜먹게 되는 맛이에용 최곱니다,고객님~ 마라쿡 입니다! 음식이 고객님 입맛에 맞아서 다행입니다! 재주문까지 해주시...,5,마라탕이 뜨거워서 먹기가 힘들었어요. 좀 더 덜어서 주셨으면 좋겠어요.,0


In [ ]:
mask = add_2_train['neg_reply'].str.len() >= 2
add_2_train[mask]

,original_review,original_reply,rating,gpt_review,neg_reply
10,배달이 빨라요 진짜로 보쌈 넘 맛있고 무랑 김치랑 잘어울려요,감사합니다. 저희 가게 족발과 보쌈은 매일 직접 삶아 정성을 더해 고객님들에게 찿아...,5,배달이 너무 늦었어요. 보쌈도 맛이 없었어요.,('배달이 지연된 것에 대해 대단히 죄송합니다. 저희 가게는 배달 시간 준수를 최우...
11,사장님 소금뭐예여?열라맛잇음,감사합니다. 저희 가게 족발은 커큐민 강황 을 함유하여 조리하고 있습니다. 밥에도 ...,5,소금이 너무 많이 들어갔어요. 입맛에 안맞았어요.,"('입맛에 맞지 않아 정말 죄송합니다. 소금 양을 조절하는 부분을 강화하여, 다음에..."
12,맛있습니다 양도많고 배달도빠릅니다강추,감사합니다. 음식은 맛도 중요하지만 빠른 배송으로 빨리 드시는 것이 음식의 맛을 살...,5,맛이 별로였어요. 배달이 오래 걸렸어요.,('맛이 별로여서 정말 죄송합니다. 고객님의 기대에 부응하지 못했던 점 양해해주시기...
13,정말 맛있었어요. 포장도 깔끔했구요.,감사합니다. 저희 가게 족발과 보쌈은 매일 새벽 재료가 들어와 당일 삶아 당일 판매...,5,맛이 별로였어요. 양도 적고 배달이 오래 걸렸어요.,('맛이 별로여서 정말 죄송합니다. 양도 충분하지 않았던 점과 배송 속도에 대하여 ...
14,배달빠르고. 맛양 모두만족,맛있게 드셨다고 하니 감사드립니다. 좋은 후기 다시 한 번 감사드리며 항상 맛있는 ...,5,맛이 별로였어요. 보쌈이 시든 느낌이 들었어요.,('맛이 별로여서 정말 죄송합니다. 보쌈에 대한 만족도를 최상으로 유지하고자 항상 ...
...,...,...,...,...,...
985,맛있어요 매장가서 먹어보고 자주 시켜먹어요~,안녕하세요 리뷰말씀 남겨 주셔서 진심으로 감사의 말씀 드립니다 언제라도 부담없이 ...,5,양이 너무 적고 서비스도 별로였어요.,안녕하세요! 1985년에 양이 너무 적고 서비스도 별로였다고 말씀하셨네요. 저희 가...
986,처음인데 양도푸짐하고 맛있고 빨리 받았습니다 .,많고 많은 가게들 중 저희 가게를 선택하고 이용해주신 고객님들을 위해 내 가족이 먹...,5,맛도 별로였고 배달이 너무 느렸어요.,안녕하세요! 1986년에 맛이 별로였고 배달이 너무 느렸다고 말씀하셨네요. 저희 가...
987,배달시간보다 너~어무 빨리와서 놀랬어용. 예전에 포장해와서 너무 맛있었던 기억에 주...,안녕하세요! 주문해주셔서 정말 감사드립니다! 앞으로도 더욱 좋은 음식과 더욱 맛있는...,5,음식이 맛이 없고 퀄리티도 별로였어요.,안녕하세요! 1987년에 음식의 맛과 퀄리티가 별로였다고 말씀하셨네요. 저희 가게에...
988,식후감 우와~ 양이 푸짐하니 지~인짜 많아요! 양도 많은데 맛도 너무 좋아요! 재...,반가워요! 사진 리뷰 너무너무 감사해요~ 맛있다고 해주셔서 감동의 눈물을 흘리고 있...,5,맛이 너무 싱겁고 기대한 것보다 별로였어요.,안녕하세요! 1988년에 맛이 싱겁고 기대한 것보다 별로였다고 말씀하셨네요. 저희 ...


In [ ]:
add_1_train[mask].to_csv(f'add_complete_1_train.csv', index=False)

In [ ]:
add_1_train[~mask].to_csv(f'add_towork_1_train.csv', index=False)

In [ ]:
eleven_reply.columns = ['reply']

In [ ]:
total_list = []
for i, lst in enumerate(eleven_reply['reply']):
    sentence = lst.split('\n')
    total_list.append(sentence)



In [ ]:
total_list = []
for i, lst in enumerate(eleven_reply['reply']):
    sentence = lst.split('"')
    total_list.append(sentence)


In [ ]:
total_list

[['[',
  '죄송합니다. 맛에 대한 실망을 드린 점 깊이 반성하겠습니다. 저희 음식점은 항상 고객님들께 맛있는 음식을 제공하기 위해 노력하고 있는데, 이번에는 맛이 별로였다는 점으로 인해 실망스러우셨을 것으로 생각됩니다. 앞으로는 더욱 향상된 맛과 촉촉함으로 만족스러운 음식을 제공하도록 노력하겠습니다. 다음에는 만족스러운 경험을 제공할 수 있도록 노력하겠습니다.',
  ',\n',
  '저희 음식점의 제육 요리에 대한 맛에 대한 실망을 드려서 죄송합니다. 저희 음식점은 다양한 메뉴를 제공하고 있으니 다음에는 다른 음식을 시켜보시는 것을 추천드립니다. 맛있는 음식으로 보답해 드리기 위해 계속해서 노력하겠습니다. 감사합니다.',
  ',\n',
  '고객님의 리뷰를 읽고 너무 놀랐습니다. 배달 시간이 오래 걸린 부분에 대해 진심으로 사과드립니다. 신속한 배송을 위해 더욱 노력하도록 하겠습니다. 저희 음식점에서는 식사를 하실 때 항상 따뜻한 음식을 선호하실 것으로 생각합니다. 향기롭고 따뜻한 음식을 조금 더 빠르게 제공할 수 있도록 노력하겠습니다. 앞으로는 더 맛있는 음식과 합리적인 배달 시간으로 고객님에게 보답하겠습니다.',
  ',\n',
  '음식의 짭짤함으로 인해 입맛에 맞지 않았다는 점 정말 죄송합니다. 우리 음식점은 어떤 요리를 준비하더라도 고객님들께서 맛을 최대한 즐기실 수 있도록 맛에 신경을 많이 쓰고 있습니다. 덜 짠 음식을 선호하신다는 점을 감안하여 앞으로는 훨씬 덜 짠 음식을 제공하는데 노력하겠습니다. 다음 번에는 만족스러운 음식을 제공할 수 있도록 노력하겠습니다.',
  ',\n',
  '라떼 앗집에서는 항상 맛있는 음료와 와플을 제공해드리기 위해 노력하고 있는데, 단맛에 대한 불편을 드렸다니 정말 죄송합니다. 단맛은 개인의 취향에 따라 다를 수 있으나, 더욱 향긋하고 부드러운 라떼를 제공할 수 있도록 노력하겠습니다. 앞으로도 항상 맛있는 음식과 음료로 고객님을 만족시킬 수 있는 서비스를 제공할 것을 약속드립니다.',
  ',\n',
  

In [ ]:
df = pd.DataFrame(pd.DataFrame(total_list).iloc[:,:10].to_numpy().reshape(-1,1))
df.columns = ['neg_reply']

In [ ]:
eleven_train.drop(columns= 'neg_reply',axis=1,inplace=True)

In [ ]:
eleven_reply

,reply
0,"[""죄송합니다. 맛에 대한 실망을 드린 점 깊이 반성하겠습니다. 저희 음식점은 항상..."
1,"1. ""죄송합니다. 맛에 만족을 드리지 못한 것 같아 정말 유감입니다. 다음에는 더..."
2,"""고객님께서는 음식의 맛과 촉감에 대해 불만을 표시해주셨는데, 이는 많이 실망스러워..."
3,"1. ""저희 음식점에서 실망을 드려 죄송합니다. 어떤 점에서 불만을 느끼셨는지 알려..."
4,"""고객님께서 음식이 맛이 별로라고 하셨군요. 저희 음식점은 맛과 품질에 항상 신경을..."
...,...
95,"""맛이 별로이셨다니 정말 죄송합니다. 다음에는 맛있는 음식으로 보답드릴게요, "", ..."
96,"""죄송합니다. 음식이 촉촉하지 않고 식어서 불편하셨을 것 같아요. 이번에는 더 신경..."
97,"""죄송합니다. 음식이 별로인 만큼 다음에는 더욱 노력하도록 하겠습니다."", ""이해하..."
98,"""제공한 음식이 맛이 별로셨다는 점, 음식의 촉촉함이 부족하여 식어서 먹기가 불편했..."


In [ ]:
eleven_train

,Unnamed: 0,review,reply,rating,sentiment,score,neg_review
0,7,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"저희 메뉴 맛있죠? 이렇게 빠져 드시는 거에요 . 그냥 일로서 만드는 게 아니라, ...",5,1,0.686599,"('맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요.. 다음에는 ..."
1,37,제육 지이인짜 맛있네요! 불맛나고 재주문의사잇어요! 서비스 묵사발도 감사합니다~,이렇게 저희 매장 찾아주셔서 감사드립니다 준비해드린 메뉴와 서비스 모두 잘 드셔 주...,5,1,0.686460,"('제육은 딱히 맛있지 않아서 실망스러웠어요. 다른 음식을 시켜야겠어요.', '')"
2,113,맛있어요 또 주문할게요,"행복한 시간 보내셨을까요? 항상 생각하지만, 저희 매장을 찾아 주시고 맛있다고 말씀...",5,1,0.695989,('배달이 너무 오래 걸려서 음식이 완전히 식었어요. 이 정도로 기다릴 가치가 없어...
3,124,배달도 빠르고 음식도 맛있어요!,안녕하세요 이렇게 뵙게 되어 너무 반갑습니다~! 저희 매장 믿고 찾아주셔서 너무 감...,5,1,0.699273,"('음식이 너무 짜서 입맛에 맞지 않았어요. 좀 덜 짰으면 좋겠어요.', '')"
4,156,맛있네요 배달도 빠르고,"저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 신선한 재료, 열심히 준비한 ...",5,1,0.697996,('라떼 앗집은 항상 맛있게 만들어주지만 와플은 너무 달아서 먹기가 힘들었어요. 단...
...,...,...,...,...,...,...,...
995,32857,오랜만에 먹었는데 여전히 맛있네요 롱쓰 흥해라!,오랜만에 반가웠습니다 응원 받아서 더 많이 드릴게요. 언제나 맛있는 식사 해드릴게요...,5,1,0.685939,"('', '원래 좋아하는 음식이었는데 맛이 너무 실망스러웠어요')"
996,32866,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,1,0.699261,"('', '완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요')"
997,33006,언제나 믿고 먹는 롱쓰에요. 짬뽕 메뉴도 처음 시켜봤는데 얼큰하고 괜찮네요. 또 주...,넿 감사합니다 믿고 시켜주신다니 너무 감사합니다. 언제나 보답하는 마음으로 맛있게 ...,5,1,0.691650,"('', '음식이 더운 상태로 배달되어서 식어서 먹을 수가 없었습니다. 맛도 별로였..."
998,33069,멘보샤 정말맛있고 새우많이들었어요 크기도커요,저희 도야짬뽕 계림점을 찾아 주셔서 감사합니다! 소중한 사진과 함께 리뷰까지 남겨주...,5,1,0.698842,"('', '다른 음식점 주문해서 먹을걸 그랬어요. 정말 실망했습니다')"


In [ ]:
train = pd.concat([eleven_train,df],axis=1)

In [ ]:
train

,Unnamed: 0,review,reply,rating,sentiment,score,neg_review,neg_reply
0,7,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"저희 메뉴 맛있죠? 이렇게 빠져 드시는 거에요 . 그냥 일로서 만드는 게 아니라, ...",5,1,0.686599,"('맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요.. 다음에는 ...","[""죄송합니다. 맛에 대한 실망을 드린 점 깊이 반성하겠습니다. 저희 음식점은 항상..."
1,37,제육 지이인짜 맛있네요! 불맛나고 재주문의사잇어요! 서비스 묵사발도 감사합니다~,이렇게 저희 매장 찾아주셔서 감사드립니다 준비해드린 메뉴와 서비스 모두 잘 드셔 주...,5,1,0.686460,"('제육은 딱히 맛있지 않아서 실망스러웠어요. 다른 음식을 시켜야겠어요.', '')","""저희 음식점의 제육 요리에 대한 맛에 대한 실망을 드려서 죄송합니다. 저희 음식점..."
2,113,맛있어요 또 주문할게요,"행복한 시간 보내셨을까요? 항상 생각하지만, 저희 매장을 찾아 주시고 맛있다고 말씀...",5,1,0.695989,('배달이 너무 오래 걸려서 음식이 완전히 식었어요. 이 정도로 기다릴 가치가 없어...,"""고객님의 리뷰를 읽고 너무 놀랐습니다. 배달 시간이 오래 걸린 부분에 대해 진심으..."
3,124,배달도 빠르고 음식도 맛있어요!,안녕하세요 이렇게 뵙게 되어 너무 반갑습니다~! 저희 매장 믿고 찾아주셔서 너무 감...,5,1,0.699273,"('음식이 너무 짜서 입맛에 맞지 않았어요. 좀 덜 짰으면 좋겠어요.', '')","""음식의 짭짤함으로 인해 입맛에 맞지 않았다는 점 정말 죄송합니다. 우리 음식점은 ..."
4,156,맛있네요 배달도 빠르고,"저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 신선한 재료, 열심히 준비한 ...",5,1,0.697996,('라떼 앗집은 항상 맛있게 만들어주지만 와플은 너무 달아서 먹기가 힘들었어요. 단...,"""라떼 앗집에서는 항상 맛있는 음료와 와플을 제공해드리기 위해 노력하고 있는데, 단..."
...,...,...,...,...,...,...,...,...
995,32857,오랜만에 먹었는데 여전히 맛있네요 롱쓰 흥해라!,오랜만에 반가웠습니다 응원 받아서 더 많이 드릴게요. 언제나 맛있는 식사 해드릴게요...,5,1,0.685939,"('', '원래 좋아하는 음식이었는데 맛이 너무 실망스러웠어요')",
996,32866,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,1,0.699261,"('', '완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요')","""고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기..."
997,33006,언제나 믿고 먹는 롱쓰에요. 짬뽕 메뉴도 처음 시켜봤는데 얼큰하고 괜찮네요. 또 주...,넿 감사합니다 믿고 시켜주신다니 너무 감사합니다. 언제나 보답하는 마음으로 맛있게 ...,5,1,0.691650,"('', '음식이 더운 상태로 배달되어서 식어서 먹을 수가 없었습니다. 맛도 별로였...",
998,33069,멘보샤 정말맛있고 새우많이들었어요 크기도커요,저희 도야짬뽕 계림점을 찾아 주셔서 감사합니다! 소중한 사진과 함께 리뷰까지 남겨주...,5,1,0.698842,"('', '다른 음식점 주문해서 먹을걸 그랬어요. 정말 실망했습니다')","""진심으로 죄송합니다. 맛과 배달 속도에 대한 실망을 드린 점 정말 죄송합니다. 저..."


In [ ]:
mask = (train['neg_review'].str.strip().str.lower() == '') | (train['neg_reply'].str.strip().str.lower() == 'none')
train[mask]

,Unnamed: 0,review,reply,rating,sentiment,score,neg_review,neg_reply


In [ ]:
train.replace('None', 0)

,review,reply,rating,sentiment,score,neg_review,neg_reply
0,진짜 돈까스 너무 맛있어요 . 진짜로,이용해주시고 맛있게드셔주셔서 감사드립니다! 즐거운 하루 보내세요!~,5,1,0.865275,"('', '맛이 별로였어요.')","[""죄송합니다. 맛이 별로였다니 정말 안타깝네요. 다음에는 더 좋은 맛으로 보답하겠..."
1,너무 맛있게 잘 먹었습니다,이용해주시고 맛있게드셔주셔서 감사드립니다! 즐거운 하루 보내세요!~,5,1,0.769657,"('', '음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요.. 다음에는 안시킬 거 ...","""촉촉하지 않고 식어서 식사가 불편했던 것 같습니다. 앞으로는 더 신경 써서 온도 ..."
2,손이 바빠집니다 입도 바빠집니다 너무 푸짐합니다 진심 맛있습니다,이용해주시고 정성가득하고 기분좋은 포토리뷰 너무 감사합니다! 즐거운 저녁시간 보내세...,5,1,0.864457,"('', '손님을 기다리는 시간이 너무 길었어요.')","""손님을 기다리게 해서 죄송합니다. 앞으로는 더 신속하게 응대하도록 노력하겠습니다."","
3,예쁜 누나인지 잘생긴 형아인지 모르겠지만 밥 많이 주셔서 오늘 하루가 배부르고 기분...,예쁜 누나 일듯요~ 이용해주시고 그분좋은 포토 리뷰 너무감사합니다! 즐거운 저녁시간...,5,1,0.724625,"('', '배달이 너무 늦게 왔네요. 서비스가 아쉬워요')","""배달이 늦었던 점 정말 죄송합니다. 더욱 신속한 서비스로 보답하겠습니다."","
4,일하는 직장에서 너무배고파서 배달시켰네요 단무지랑 김치안먹어서 빼고 혹시 밥좀더주실...,저는 잘드셨다는 리뷰만봐도 배가부릅니다! 오늘도 화이팅 하시고 즐거운 하루 보내세요!~,5,1,0.910658,"('', '메뉴가 다양하지 않아서 선택이 어려웠어요.')","""메뉴의 다양성이 부족했던 걸로 알고 있습니다. 더 다양한 메뉴를 추가할 수 있도록..."
...,...,...,...,...,...,...,...
995,대전에서 배달 초밥집은 여기임,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.742414,"('맛이 별로였어요. 배달이 늦고 음식도 식어서 먹을 수 없었어요', '')","6. ""맛이 싱겁고 별로였다니 정말 안타깝습니다. 저희 음식이 고객님께 실망스러운 ..."
996,정말 상무초밥이 근처에 있어서 너무 감사한 마음뿐이에요. 정말 너무 좋습니다!,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.926462,"('상무 초밥은 너무 싱겁고 맛이 없어요. 실망이었어요', '')","7. ""음식의 맛과 식감이 좋지 않았다니 정말 죄송합니다. 저희 음식에 불만을 느끼..."
997,항상 시켜 먹을 때마다 너무 맛있어요!,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.823359,"('음식이 별로였어요. 맛이 없고 식감도 좋지 않아요', '')","8. ""맛이 별로였고 식감도 좋지 않았다니 정말 안타깝습니다."
998,맛나요 정신없이 먹느라 중간에 사진 찍어요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.713090,"('맛이 별로였어요. 식감도 불편하고 배달이 너무 늦어서 실망이었어요', '')",None


In [ ]:
mask = (train['neg_reply'] == "")
train[mask]

,Unnamed: 0,review,reply,rating,sentiment,score,neg_review,neg_reply
1,6660,너~무 맛있게 잘 먹었어요 배가 부른데도 손에서 젓가락을 놓지 못하겠더라구요 간도 ...,항상 웃음이 가득한 구구족을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각...,5,1,0.906428,"('', '가격 대비 맛이 별로예요. 다른 가게에서 시키는게 낫겠어요')",
3,6668,항상 여기서 시켜요! 오늘도 맛있습니다!,식사는 맛있게 하셨나요? 혜화 대학로 SNS 피자랭킹맛집 피자보이시나 대학로점 을...,5,1,0.709532,"('', '음식이 맛이 별로였어요. 좀 더 맛있는 곳을 찾아봐야 겠어요')",
5,6674,일주일에 한번은 피자 먹는 사람이고 내가 피자 좋아하는거의 베이스는 빵을 좋아하는 ...,식사는 맛있게 하셨나요? 혜화 대학로 SNS 피자랭킹맛집 피자보이시나 대학로점 을...,5,1,0.883377,"('', '음식이 맛이 별로 없었어요. 좀 실망스러웠어요')",
7,6679,예상시간보다 훨씬 빨리 가져다 주셨어요~ 피자쏘스 찐하고 양도 토핑양도 넉넉히 얹어...,식사는 맛있게 하셨나요? 혜화 대학로 SNS 피자랭킹맛집 피자보이시나 대학로점 을...,5,1,0.903428,"('', '다른 리뷰들보다 맛이 별로였어요. 기대하던 만큼이 아니었어요')",
9,6682,항상 맛있게 먹고 있습니다 리뷰이벤트오 주신 스파게티도 너무 맛있네요~,식사는 맛있게 하셨나요? 혜화 대학로 SNS 피자랭킹맛집 피자보이시나 대학로점 을...,5,1,0.794446,"('', '음식이 맛이 별로였어요. 가격 대비 맛이 아쉬웠어요')",
...,...,...,...,...,...,...,...,...
981,10155,사진을 못찍었네요 정말 맛있어요! 항상 여기서만 먹고있어요,안녕하세요.삽교원조두리곱창입니다. 코로나로인해 고객님도 어려운시기에 찾아주셔서 감사...,5,1,0.852417,"('여기 음식은 별로예요, 맛이 없어서 실망했어요', '')",
983,10164,기존까지는 먹었던 돼지곱창이 일반 음식점에서 먹은 것이라면 여기는 돼지곱창 전문점이...,안녕하세요.삽교원조두리곱창입니다. 코로나로인해 고객님도 어려운시기에 찾아주셔서 감사...,5,1,0.807200,"('음식이 매우 싱겁고 맛이 없어서 실망했어요', '')",
985,10167,앞으로 곱창 막창은 여기서만 시킬 것 같아요! 너무너무너무 마시써요,안녕하세요.삽교원조두리곱창입니다. 코로나로인해 고객님도 어려운시기에 찾아주셔서 감사...,5,1,0.873805,"('막창과 곱창이 맛없어서 실망했어요, 다음에는 다른 음식을 시켜야겠어요', '')",
987,10175,진짜 우리동네에서 제일 맛있음,안녕하세요.삽교원조두리곱창입니다. 코로나로인해 고객님도 어려운시기에 찾아주셔서 감사...,5,1,0.848467,"('막창이 맛이 없고 불쾌한 맛이 났어요, 정말 실망스러웠어요', '')",


In [ ]:
train[mask]

,Unnamed: 0,review,reply,rating,sentiment,score,neg_review,neg_reply
21,856,당치땡처음인데 너무 맛있어요!,안녕하세요! 당신은 지금 치킨이 땡긴다 헬리오시티점 입니다 첫주문 너무 감사드려요!...,5,1,0.699440,"('별로 맛없었어요. 특히 갈비튀김은 식어서 먹을 수도 없었어요', '')",
23,896,갈비튀김이라니! 엄청 부드럽고 튀김옷이 바삭해서 맛있네요 서비스도 감사합니다~,항상 친절한 응대로 기분까지 챙길수있도록 하겠습니다 별5점리뷰 너무 감사드려요 제...,5,1,0.697137,"('기대했던 만큼 맛있지 않았어요. 다음에는 다른 메뉴를 시키게 될 것 같아요', '')",
25,954,처음 시켜보는데 맛있네요 치킨 이제 여기서 시킬듯,안녕하세요 당치땡 헬리오시티점 입니다 항상 친절한 응대로 기분까지 챙길수있도록 하겠...,5,1,0.685106,"('치킨을 시켰는데, 내용물이 많이 부족했어요. 좀 실망스러웠어요', '')",
27,1017,음식도 포장도 깔끔하고 배달이 좀 늦게 왔는데 친절하게 응대해주셨어요 맛있게 먹었습니다,"맛있게 잘 드셔 주셨다니, 저희도 너무 기쁘네요 앞으로도 치킨이 땡길때엔 당치땡입니...",5,1,0.695955,"('음식이 맛있지 않았어요. 별로였어요. 다음에는 다른 곳에서 시킬 거 같아요', '')",
29,1066,매번맛있게 먹고있는 집이에요~ 식사에도 해장에도 최고입니당~,"al 님, 사랑하는 고객니이이이임! 매번 맛있게 드셔주시고 오늘도 또 찾아주시니 정...",5,1,0.698223,"('기대반의구심 반으로 시켰는데, 생각보다 맛이 없었어요. 너무 실망이에요', '')",
...,...,...,...,...,...,...,...,...
991,32727,맛있게 잘 먹었습니다 또 주문할게요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.689060,"('', '전혀 맛이 없었어요. 진짜 실망했어요')",
993,32774,맛잇습니다! 또시켜먹을께요 모밀은 시켜먹어본것중 제일 맛잇습니다,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.690947,"('', '음식이 맛이 없고, 냄새도 좀 나서 정말 별로였습니다')",
995,32857,오랜만에 먹었는데 여전히 맛있네요 롱쓰 흥해라!,오랜만에 반가웠습니다 응원 받아서 더 많이 드릴게요. 언제나 맛있는 식사 해드릴게요...,5,1,0.685939,"('', '원래 좋아하는 음식이었는데 맛이 너무 실망스러웠어요')",
997,33006,언제나 믿고 먹는 롱쓰에요. 짬뽕 메뉴도 처음 시켜봤는데 얼큰하고 괜찮네요. 또 주...,넿 감사합니다 믿고 시켜주신다니 너무 감사합니다. 언제나 보답하는 마음으로 맛있게 ...,5,1,0.691650,"('', '음식이 더운 상태로 배달되어서 식어서 먹을 수가 없었습니다. 맛도 별로였...",


In [ ]:
mask = (train['neg_review'].str.len() <5) | (train['neg_reply'].str.len() <10) | (train['neg_reply'] == "") | (train['neg_reply'] == " ")
filtered_train = train[~mask]

In [ ]:
filtered_train

,Unnamed: 0,review,reply,rating,sentiment,score,neg_review,neg_reply
0,7,저도 모르게 은근 여기 자주 시켜먹고 있었네요 늘 잘먹고 있습니다,"저희 메뉴 맛있죠? 이렇게 빠져 드시는 거에요 . 그냥 일로서 만드는 게 아니라, ...",5,1,0.686599,"('맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요.. 다음에는 ...","[""죄송합니다. 맛에 대한 실망을 드린 점 깊이 반성하겠습니다. 저희 음식점은 항상..."
1,37,제육 지이인짜 맛있네요! 불맛나고 재주문의사잇어요! 서비스 묵사발도 감사합니다~,이렇게 저희 매장 찾아주셔서 감사드립니다 준비해드린 메뉴와 서비스 모두 잘 드셔 주...,5,1,0.686460,"('제육은 딱히 맛있지 않아서 실망스러웠어요. 다른 음식을 시켜야겠어요.', '')","""저희 음식점의 제육 요리에 대한 맛에 대한 실망을 드려서 죄송합니다. 저희 음식점..."
2,113,맛있어요 또 주문할게요,"행복한 시간 보내셨을까요? 항상 생각하지만, 저희 매장을 찾아 주시고 맛있다고 말씀...",5,1,0.695989,('배달이 너무 오래 걸려서 음식이 완전히 식었어요. 이 정도로 기다릴 가치가 없어...,"""고객님의 리뷰를 읽고 너무 놀랐습니다. 배달 시간이 오래 걸린 부분에 대해 진심으..."
3,124,배달도 빠르고 음식도 맛있어요!,안녕하세요 이렇게 뵙게 되어 너무 반갑습니다~! 저희 매장 믿고 찾아주셔서 너무 감...,5,1,0.699273,"('음식이 너무 짜서 입맛에 맞지 않았어요. 좀 덜 짰으면 좋겠어요.', '')","""음식의 짭짤함으로 인해 입맛에 맞지 않았다는 점 정말 죄송합니다. 우리 음식점은 ..."
4,156,맛있네요 배달도 빠르고,"저희 매장 찾아주신 것도 감사한데 리뷰까지.감동입니다 신선한 재료, 열심히 준비한 ...",5,1,0.697996,('라떼 앗집은 항상 맛있게 만들어주지만 와플은 너무 달아서 먹기가 힘들었어요. 단...,"""라떼 앗집에서는 항상 맛있는 음료와 와플을 제공해드리기 위해 노력하고 있는데, 단..."
...,...,...,...,...,...,...,...,...
990,32714,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.694974,"('', '맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요.. 다...","""안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망..."
992,32739,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,1,0.697194,"('', '맛도 없고, 배달도 늦어서 너무 실망했습니다')","""고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요..."
994,32847,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,1,0.692698,"('', '진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요')","""맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으..."
996,32866,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,1,0.699261,"('', '완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요')","""고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기..."


In [ ]:
filtered_train.to_csv('filtered_final_train_11000_12000.csv',index=False)

# 추가 크롤링

In [ ]:
gpt_aug_num0 = pd.read_csv("filtered_final_train_1_1000.csv") #850
gpt_aug_num1 = pd.read_csv("filtered_final_train_1000_2000.csv") #860
# gpt_aug_num2 = pd.read_csv("filtered_final_train_2000_3000.csv") #820
gpt_aug_num3 = pd.read_csv("filtered_final_train_3000_4000.csv") #950
gpt_aug_num4 = pd.read_csv("filtered_final_train_4000_5000.csv") #940
gpt_aug_num5 = pd.read_csv("filtered_final_train_5000_6000.csv") #920


In [ ]:
train = pd.read_csv('gpt_neg_9400.csv')
train

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9373,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9374,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9375,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9376,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [ ]:
mask = train

In [ ]:

review_list = []
for review in train['gpt_review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣.,~?!]") # 영어 제거
        result = pat.sub(" ", review)

        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['gpt_review'] = review_list
train.head()


,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585


In [ ]:
reply_list = []
for reply in train['gpt_reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣.,~?!]") # 영어 제거
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^[.]\s*', '', result, count=1)
        result = re.sub(r'^[. ]+', '', result, count=1)

        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['gpt_reply'] = reply_list
train.head()


,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585


In [ ]:
train['gpt_reply']

0       안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...
1       배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...
2       음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...
3       잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...
4       가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...
                              ...                        
9373    안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...
9374    고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....
9375    맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...
9376    고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...
9377    진심으로 죄송합니다. 맛과 배달 속도에 대한 실망을 드린 점 정말 죄송합니다. 저희...
Name: gpt_reply, Length: 9378, dtype: object

In [ ]:
train

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9373,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9374,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9375,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9376,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [ ]:
mask = train['gpt_reply'].str.contains('첫째, |둘째, |셋째, |넷째, |다섯째, |여섯째, |일곱째, |여덟째, |아홉째, |열째, ')
train[mask]

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
7507,배송도 빠르고 양도 최고 맛도 최고예요!,"식사 맛있게 하셨나욤 저희가 준비한 음식 맛있게 드셔주시고, 리뷰 남겨주셔서 감사드...",5,배송이 너무 늦어서 기다리느라 짜증났어요. 다음에는 다른 곳에서 시킬게요,"첫째, 죄송합니다. 배송이 늦어서 기다리느라 짜증을 드렸군요. 다음에는 더 빠른 배...",0,0.819173
7508,왕 맛있어요! 양 많구 굿!,"안녕하세요 소중한 주문해주시고, 리뷰남겨주셔서 감사해요 다음에도 비빔밥 생각나시면 ...",5,맛이 너무 싱거웠어요. 한 입 먹고 소화가 안되네요. 맛이 좀 아쉬웠어요,"둘째, 맛이 싱거웠다니 정말 죄송합니다. 다음에는 맛에 더 신경쓰도록 노력하겠습니다.",0,0.929940
7509,아니. 이게 무슨 일이죠? 왜이렇게 맛있는거죠! 심지어 양도 엄청 많아요! 먹다가 ...,식사는 맛있게 하셨나요 저희 음식점 찾아주셔서 정말 감사해요 ! 언제든지 저희 음식...,5,이 가격에 이런 음식을 먹다니 정말 실망스러웠어요. 돈 주고서 이런 걸 먹다니.,"셋째, 가격에 비해 실망스러웠다니 참 안타까워요. 다음에는 조금 더 만족스러운 음식...",0,0.907507
7510,오랜만에 주문했는데 역시 맛있네요 밥양이 적은게 아닌데 토핑이 풍부해서 주문할때마다...,"고객님 안녕하세요 ! 저희 음식점을 찾아주시고, 맛있게 드셔주셔서 감사합니당 앞으...",5,양이 너무 적어서 배고프게 먹었어요. 빵 한 조각 보단 나았던 거 같아요,"넷째, 양이 부족해서 배고프셨다니 죄송합니다. 기회를 주신다면 보다 풍성하고 맛있는...",0,0.902548
7511,양도 푸짐하고 맛도 최고예요!,안녕하세요 ! 저희가 준비한 음식을 맛있게 드셔주셔서 정말 감사합니다 다음에도 맛...,5,맛이 좀 짜서 먹기가 힘들었어요. 좀 덜 짜게 해주셔야 할 것 같아요,"다섯째, 맛이 짜서 드셔야 하는 데 힘드셨다니 정말 죄송합니다. 앞으로는 고객님의 ...",0,0.892063
7512,육회물회랑 육회비빔밥 시킬까 고민하다가 육회물회2개로 시켰는데 배달 도착하고 냉면사...,"고객님 안녕하세요 ! 맛있게 드셔주시고, 리뷰 남겨주셔서 감사합니당 앞으로도 초심 ...",5,"분명히 맛이 좋다는 평을 들었는데, 제 입맛에는 맞지 않았어요. 간이 너무 신맛이 ...","여섯째, 맛이 맞지 않아 실망하셨다니 정말 안타깝습니다. 간을 신맛으로 드리지 않도...",0,0.903951
7513,냉면이 먹고싶을땐 여기서 먹어요. 전에 한번 먹고 맛있어서 또 시카려고 했는데 그럴...,믿고 찾아주셔서 감사합니다 ! 언제든지 재주문해 주셔도 가성비 짱 이름처럼 대왕 만...,5,서비스가 별로였어요. 배달원이 무례해서 좀 싫었어요,"일곱째, 서비스가 별로였다니 죄송합니다. 앞으로는 더 예의를 갖춘 배달원을 고용하여...",0,0.910777
7514,육회가 진짜 듬뿍이라 너무좋아요 비빔밥은 여기서만먹습니다 오늘도 잘먹었습니다!,"안녕하세요 소중한 주문해주시고, 정성 듬뿍! 담긴 리뷰까지 정말 감사합니다~ 다음에...",5,음식이 너무 뜨겁게 왔어요. 조금 잘 식혀서 배달해주셨으면 좋겠어요,"여덟째, 음식이 너무 뜨겁게 도착해서 죄송합니다. 다음에는 보다 신선하고 적절한 온...",0,0.853230
7515,맛나개 잘먹었습니다. 사장님 화이팅!,안녕하세요 손님 많은 가게들이 있지만 저희 음식점 믿고 소중한 주문해 주셔서 고맙습...,5,포장이 아주 안 좋았어요. 음식이 흐트러져서 왔어요,"아홉째, 포장이 좋지 않아서 불편을 드렸다니 정말 죄송합니다. 앞으로는 더 꼼꼼히 ...",0,0.911001
7516,해외에서 오랜만에 한국에 입국해서 가족모두 밥을 먹고싶어 호텔로 주문했습니다. 예상...,안녕하세요 손님 저희 음식 맛있게 드셔주시니 오히려 저희가 더 감사드리죠! 많은 가...,5,레스토랑에서 먹는 맛과는 달라서 좀 아쉬웠어요. 배달이라 그런가요?,"열째, 레스토랑에서 먹는 맛과 다르셨다니 정말 안타까워요. 배달로 인한 차이일 수도...",0,0.892992


In [ ]:
pattern = r'첫째, |둘째, |셋째, |넷째, |다섯째, |여섯째, |일곱째, |여덟째, |아홉째, |열째, '
mask = train['gpt_reply'].str.contains(pattern)
train['gpt_reply'] = train['gpt_reply'].str.replace(pattern, '', regex=True)


In [ ]:
mask = train['gpt_reply'].str.len() > 300
train[mask].shape

(46, 7)

In [ ]:
train[~mask].shape

(9332, 7)

In [ ]:
train = train[~mask]

In [ ]:
mask = train['gpt_reply'].str.len() < 16
train[mask]

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
1520,먹으면 속도 편하고 콜라겐도 듬뿍인 배달 족발 구구족이 제일 맛있습니 디. 양과 맛...,항상 웃음이 가득한 구구족을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생각...,5,맛도 별로였고 배달이 늦었어요.,불편을 끼쳐드려 죄송합니다.,0,0.921112
1528,배달도 빠르고 아주 맛있어요! 감사합니다,항상 웃음이 가득한 구구족을 찾아주셔서 감사합니다 언제나 내가 먹는 음식이라고 생...,5,배달이 너무 늦게 왔고 음식도 식어서 별로였어요.,불편을 끼쳐드려 죄송합니다.,0,0.936983
2975,친절하고 맛있고 빠르고 감사합니다,안녕하세요 앞으로도 변함없는 맛과 정성으로 보답하는 피자이탈리 운양점이 되겠습니다 ...,5,양이 너무 적고 서비스도 별로였어요.,,0,0.923027
8025,여기 어떤날에 쉬어서 다른데도먹어봣는데 역시 여기가젤 맛잇,안녕하세요~ 고객님의 최고의 칭찬속에 만족스런 식사를 하신것 같아 저희 안녕닭 충남...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님.",0,0.942769
8027,항상 잘먹고있어요 감사합니다,안녕하세요 수 많은 맛집속에 저희를 믿고 주문 주시고 맛있게 드셔주심 진심으로 감사...,5,배달이 많이 늦어지고 음식도 차가웠어요. 좀더 신경써서 배달해주셨으면 좋았을 텐데 ...,"안녕하세요, 고객님.",0,0.909884
8029,very delicious thank you,안녕하세요! 안녕닭 충남대점입니다 잊지 않고 작성해주신 소중한 리뷰 감사드립니다! ...,5,맛이 예전보다 많이 변했어요. 이전에는 맛있게 먹었는데 이번에는 좀 실망했습니다,"안녕하세요, 고객님.",0,0.905371
8031,너무 맛있어보여서 급하게 먹느라 사진을 깜빡 잊어버렸네여. 그만큼 너무 맛있었음! ...,먼저 저희 매장을 이용해 주셔서 감사합니다! 사진리뷰와 별5개 너무나 감사드립니다!...,5,음식이 생각보다 싱겁고 부족해서 아쉬웠어요. 좀더 양과 맛에 신경써주셨으면 좋았을텐데,"안녕하세요, 고객님.",0,0.891958
9087,겁나 맛있어요 중국당면이 한 개밖에 안들어있어서 당황하긴 햌 지만,저희라화방 마라탕을 믿고 찾아주셔서 너무 감사드립니다 맛잇게 드셧다고 하시니 너무 ...,5,공기밥이 좀 딱딱해서 먹기가 좀 그랬어요. 좀 더 부드럽게 주셨으면 좋겠어요.,공기밥이 딱딱해서 좀 그랬,0,0.819961
9236,시키는 메뉴마다 항상 맛있어요 잘먹었습니다,시키는 메뉴마다 항상 맛있다니요 정말 최고의 칭찬이십니다 앞으로도 맛있는 커피와 디...,5,맛이 없어서 먹다가 그만두었어요. 좀 더 맛있는 음식을 찾아봐야겠어요,"안녕하세요, 맛이 없어",0,0.924744


In [ ]:
train = train[~mask]

In [ ]:
train[['gpt_reply']]

,gpt_reply
0,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...
1,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...
2,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...
3,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...
4,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...
...,...
9373,"안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하..."
9374,고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....
9375,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...
9376,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...


In [ ]:
train.to_csv(f'gpt_neg_9300.csv', index=False)

In [ ]:
train = pd.read_csv('gpt_neg_9300.csv')

In [ ]:
train

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,0.908812
1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,0.820514
2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,0.941290
3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,0.910234
4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,0.928585
...,...,...,...,...,...,...,...
9318,늘믿고먹어요 싱싱맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","안녕하세요, 고객님. 말씀해주신 점 먼저 진심으로 사과드립니다. 맛과 촉감에 실망하...",0,0.942769
9319,초밥은 늘 여기만 주문해요. 늘 맛나요,안녕하세요~ 상무초밥 유성점입니다~! 저희 매장을 이용해주셔서 정말 감사드립니다! ...,5,"맛도 없고, 배달도 늦어서 너무 실망했습니다",고객님께 실망을 드려 죄송합니다. 저희 음식이 맛있지 않아서 큰 실망을 받으셨군요....,0,0.912723
9320,해장하려구 시켰는데 진짜 맛있네용 . 다음에 또 시켜먹을게요 감사합니다 !,안 녕 하세요 해장은 잘 하셨어요? 속풀이에도 좋고 한잔 할때도 좋고 다 어울리지요...,5,진짜 최악이었어요. 맛도 없고 배달도 오래걸렸어요,맛이 없고 배달이 늦어서 정말 죄송합니다. 저희는 항상 맛과 배달 시간을 최우선으로...,0,0.894588
9321,늘 맛있어요~ 사진이 없어요~ 또 시킬께요~예쁜사진과 함께,감사합니다 다음에는 예쁜 사진 기대할게요. 오늘도 좋은 하루 되세요. 또 맛있게 뵙...,5,완전 별로였어요. 음식이 시큼해서 입에 맞지가 않았어요,고객님께 실망을 드려 정말 죄송합니다. 음식에 대한 맛과 냄새에 대한 불만을 제기해...,0,0.933854


In [ ]:
pattern = r'^[^가-힣]'
mask = train['gpt_reply'].str.contains(pattern)

train[mask]

,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score


In [ ]:
original_review = ". 고객님께서 맛이 별로셨다니 정말 죄송합니다. 저희 음식이 촉촉함을 잃어버린 것 같아 안타깝네요. 앞으로는 더 맛있는 음식을 제공하도록 노력하겠습니다."

# 정규 표현식을 사용하여 문장 첫 공백이나 마침표를 제거합니다.
new_sentence = re.sub(r'(^|[.])\s+', '', original_sentence)

In [ ]:
new_sentence

'고객님께서 맛이 별로셨다니 정말 죄송합니다저희 음식이 촉촉함을 잃어버린 것 같아 안타깝네요앞으로는 더 맛있는 음식을 제공하도록 노력하겠습니다.'

In [ ]:
new_review = re.sub(r'(^|[.])\s+', '', original_review, count=1)

print(new_review)

고객님께서 맛이 별로셨다니 정말 죄송합니다. 저희 음식이 촉촉함을 잃어버린 것 같아 안타깝네요. 앞으로는 더 맛있는 음식을 제공하도록 노력하겠습니다.
